In [1]:
!git clone https://github.com/LFPy/ElectricBrainSignals.git
%cd ElectricBrainSignals/
!pip install -e

C:\Users\SunRe_Admin\OneDrive\Skrivebord\master\ElectricBrainSignals


fatal: destination path 'ElectricBrainSignals' already exists and is not an empty directory.

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

-e option requires 1 argument


In [2]:
# imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import LFPy
import brainsignals.neural_simulations as ns
from brainsignals.plotting_convention import mark_subplots, cmap_v_e, simplify_axes
from brainsignals.neural_simulations import return_hay_cell
from matplotlib.collections import PolyCollection
from matplotlib.patches import Ellipse
import neuron

ns.load_mechs_from_folder(ns.cell_models_folder)
np.random.seed(12345)

In [3]:
# Create a grid of measurement locations, in (um)
grid_x, grid_z = np.mgrid[-650:651:20, -1000:1600:20]
grid_y = np.ones(grid_x.shape) * 0

sigma = 0.3  # S/m

# Define electrode parameters
grid_elec_params = {
    'sigma': sigma,      # extracellular conductivity
    'x': grid_x.flatten(),  # electrode positions
    'y': grid_y.flatten(),
    'z': grid_z.flatten(),
    'method': 'pointsource' # linesource
}

# Define electrode parameters
elec_params = {
    'sigma': 0.3,
    'x' : np.array([100.]),
    'y' : np.array([0.]),
    'z' : np.array([0.]),
    'method' : 'pointsource',
}

# Define electrode2 parameters
elec_params2 = {
    'sigma': 0.3,
    'x' : np.array([0.]),
    'y' : np.array([0.]),
    'z' : np.array([-100.]),
    'method' : 'pointsource',
}

In [4]:
def insert_synaptic_input(idx, cell, syn_scale):

    synapse_parameters = {'e': 0., # reversal potential
                          'weight': 0.002 * syn_scale, #  synapse weight
                          'record_current': True, # record synapse current
                          'syntype': 'Exp2Syn',
                          'tau1': 1, #Time constant, rise
                          'tau2': 3, #Time constant, decay
                          }
    synapse_parameters['idx'] = idx
    synapse = LFPy.Synapse(cell, **synapse_parameters)
    synapse.set_spike_times(np.array([1.]))
    return synapse, cell

In [5]:
# simulation
tstop = 10
dt = 2**-6
cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=True)

basal_idxs = cell.get_rand_idx_area_norm(section='allsec', z_max=0,
                                   z_min=-1e9, nidx=1000)
syn_idxs = basal_idxs
syn_idxs_list = [basal_idxs]
                
grid_electrode = LFPy.RecExtElectrode(cell, **grid_elec_params)

LFPs = []

for sim_idx, syn_idxs in enumerate(syn_idxs_list):
    cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=True)
    if sim_idx == len(syn_idxs_list) -1:
        comp_idx = 0
        for sec in cell.allseclist:
            for seg in sec:
                if cell.z[comp_idx].mean() > 500:
                    seg.g_pas *= 60
                comp_idx += 1       

    for syn_idx in syn_idxs:
        syn_scale = 1 / len(syn_idxs)
        syn, cell = insert_synaptic_input(syn_idx, cell, syn_scale)
    cell.simulate(rec_imem=True, rec_vmem=True)
    print("Max dV: ", np.max(np.abs(cell.vmem - cell.vmem[:, 0, None])))
    LFP = 1000 * grid_electrode.get_transformation_matrix() @ cell.imem
    LFPs.append(LFP)
    cell.__del__()

Max dV:  4.757341929487666


In [6]:
# electrode one
#--------------------------------------------------
e_r = np.linspace(1,1000,num=20, dtype='int')

eaps_1 = []
p2p_amps_1 = []


for elec_radius in e_r:
    if elec_radius > 1e-9:
        elec_params.update(
            N = np.array([[1, 0, 0]]), # surface normals
            r = elec_radius,           # contact site radius
            n = elec_radius * 100,     # datapoints for averaging
        )
    np.random.seed(1234)
    elec_1 = LFPy.RecExtElectrode(cell, **elec_params)
    LFP_1 = elec_1.get_transformation_matrix() @ cell.imem * 1000
    p2p_amp = np.max(LFP_1[0]) - np.min(LFP_1[0]) 
    p2p_amps_1.append(p2p_amp)
    eaps_1.append(LFP_1[0])

In [ ]:
# electrode two
#--------------------------------------------------
eaps_2 = []
p2p_amps_2 = []

for elec_radius in e_r:
    if elec_radius > 1e-9:
        elec_params2.update(
            N = np.array([[0, 0, 1]]), # surface normals
            r = elec_radius,           # contact site radius
            n = elec_radius * 100,      # datapoints for averaging
        )
    
    np.random.seed(1234)
    elec_2 = LFPy.RecExtElectrode(cell, **elec_params2)
    LFP_2 = elec_2.get_transformation_matrix() @ cell.imem * 1000
    p2p_amp_2 = np.max(LFP_2[0]) - np.min(LFP_2[0]) 
    p2p_amps_2.append(p2p_amp_2)
    eaps_2.append(LFP_2[0])

In [ ]:
# figure with dipole-lfp and membrane potential
def plot_grid_LFP(cell, LFP, ax, syn_idxs, scale_max=None):
    
    max_amp_elec_idx = np.argmax(np.max(np.abs(LFP), axis=1))
    max_amp_t_idx = np.argmax(np.abs(LFP[max_amp_elec_idx, :]))

    max_amp_LFP = np.max(np.abs(LFP))
    print(max_amp_LFP)
    if not max_amp_LFP == np.abs(LFP[max_amp_elec_idx, max_amp_t_idx]):
        raise RuntimeError("Wrong with chosen max value")

    LFP = LFP[:, max_amp_t_idx].reshape(grid_x.shape)
    num = 11
    levels = np.logspace(-2.3, 0, num=num)

    scale_max = np.max(np.abs(LFP)) if scale_max is None else scale_max
    
    levels_norm = scale_max * np.concatenate((-levels[::-1], levels))

    colors_from_map = [cmap_v_e(i/(len(levels_norm) - 2))
                       for i in range(len(levels_norm) - 1)]
    colors_from_map[num - 1] = (1.0, 1.0, 1.0, 1.0)
    ep_intervals = ax.contourf(grid_x, grid_z, LFP,
                                   zorder=2, colors=colors_from_map,
                                   levels=levels_norm, extend='both')

    ax.contour(grid_x, grid_z, LFP, colors='k', linewidths=(1), zorder=2,
                   levels=levels_norm)
    ax.plot(cell.x.T, cell.z.T, lw=1, c='gray')
    if len(syn_idxs) == 1:
        ms = 5
    else:
        ms = 1.5
    [ax.plot(cell.x[syn_idx].mean(), cell.z[syn_idx].mean(), marker='o', c='b',
                 ms=ms, mec='none', mew=0.2)
     for syn_idx in syn_idxs]
    
    return ep_intervals
scale_max = 1
ax_lfp_dict = dict(aspect=1, frameon=False, xticks=[], yticks=[],
                   ylim=[np.min(grid_z) - 2, np.max(grid_z) + 2],
                   xlim=[np.min(grid_x) - 2, np.max(grid_x) + 2])
    
plt.close("all")
fig = plt.figure(figsize=[6, 3.8])
fig.subplots_adjust(bottom=-0.05, top=1.08, right=1.03,
                    left=-0.03, wspace=-0.02, hspace=-0.1)
num_cols = 5
num_rows = 2
ax_jump = 0

imgs = []
axes_to_mark = []
for i, syn_idxs in enumerate(syn_idxs_list):
    syn_scale = scale_max
    if len(syn_idxs) > 1:
        ax_jump = 2
    ax = fig.add_subplot(num_rows, num_cols, i + 1 + ax_jump, **ax_lfp_dict)
    axes_to_mark.append(ax)    
    img = plot_grid_LFP(cell, LFPs[i], ax, syn_idxs, scale_max=syn_scale)
    imgs.append(img)

cax = fig.add_axes([0.6, 0.55, 0.01, 0.42], frameon=False)
cbar = fig.colorbar(imgs[-1], cax=cax)
cbar.set_label('V$_{\mathrm{e}}$ (µV)', labelpad=4)
cbar.set_ticks(np.array([-1, -0.1, -0.01, 0.01, 0.1, 1]) * scale_max)
bax = fig.add_axes([0.1, 0.65, 0.3, 0.2], xlabel="time (ms)", ylabel=r"$V_{\rm m}$ (mV)")
bax.plot(cell.tvec, cell.vmem[0])
mark_subplots(bax, 'A', ypos=1.25)
mark_subplots(ax, 'B', ypos=0.7, xpos=0.1)

fig_folder = 'figure_two_final'
fig_name = 'dipole'
fig.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}.png', bbox_inches = "tight")

In [ ]:
plt.semilogy(e_r, p2p_amps_1, label='electrode 1')
plt.semilogy(e_r, p2p_amps_2, label='electrode 2')
plt.xlabel('electrode radius (µm)')
plt.ylabel('peak-to-peak amplitude (µV)')
plt.legend()
fig_folder = 'figure_two_final'
fig_name = 'log_p2p_amp2_new_radii'
plt.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}.png', bbox_inches = "tight")

In [ ]:
# Plot for ecps, normalized and unnormalized
fig = plt.figure(figsize=[12, 5])

elec_clrs = {r: plt.cm.Reds(0.1 + i / (len(e_r) - 1))
             for i, r in enumerate(e_r)}

# electrode 1
ax_eap_1 = fig.add_axes([0.45, 0.7, 0.24, 0.2],
                      xlabel="time (ms)", ylabel="µV")
ax_eap_1.set_title('electrode 1')
mark_subplots(ax_eap_1, 'A')
ax_eap_norm_1 = fig.add_axes([0.45, 0.4, 0.24, 0.2],
                           xlabel="time (ms)", ylabel="normalized")

mark_subplots(ax_eap_norm_1, 'B')

# electrode 2
ax_eap_2 = fig.add_axes([0.75, 0.7, 0.24, 0.2],
                      xlabel="time (ms)", ylabel="µV")
ax_eap_2.set_title('electrode 2')
mark_subplots(ax_eap_2, 'C')
ax_eap_norm_2 = fig.add_axes([0.75, 0.4, 0.24, 0.2],
                           xlabel="time (ms)", ylabel="normalized")
mark_subplots(ax_eap_norm_2, 'D')


for i, elec_radius in enumerate(e_r):
    c = elec_clrs[elec_radius]
    l, = ax_eap_1.plot(cell.tvec, eaps_1[i], c=c)
    ax_eap_norm_1.plot(cell.tvec, eaps_1[i] / np.max(np.abs(eaps_1[i])), c=c)
    
    l, = ax_eap_2.plot(cell.tvec, eaps_2[i], c=c)
    ax_eap_norm_2.plot(cell.tvec, eaps_2[i] / np.max(np.abs(eaps_2[i])), c=c)
    
fig_folder = 'figure_two_final'
fig_name = 'ECPs_el1_el2_new_radii'
fig.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}.png', bbox_inches='tight')

In [ ]:
# final figure
elec_clrs = {r: plt.cm.Reds(0.1 + i / (len(e_r) - 1))
             for i, r in enumerate(e_r)}

plt.close("all")

fig = plt.figure(figsize=[9, 6.2])

fig.subplots_adjust(left=0.14, bottom=0.18, top=0.85, right=0.98,
                    hspace=0.2)

ax_morph = fig.add_axes([0.1, 0.03, 0.2, 0.5], xlabel='            electrode 1',frameon=False, aspect=1,
                        xticks=[], yticks=[], xlim=[-1000, 1000],
                        ylim=[-1400, 1000])

ax_morph_2 = fig.add_axes([0.3, 0.03, 0.2, 0.5], xlabel='   electrode 2', frameon=False, aspect=1,
                        xticks=[], yticks=[], xlim=[-1000, 1000],
                        ylim=[-1400, 1000])

zips = []
for x, z in cell.get_pt3d_polygons():
    zips.append(list(zip(x, z)))
polycol = PolyCollection(zips, edgecolors='none',
                         facecolors='black', zorder=100, rasterized=False)
ax_morph.add_collection(polycol)

zips = []
for x, z in cell.get_pt3d_polygons():
    zips.append(list(zip(x, z)))
polycol_2 = PolyCollection(zips, edgecolors='none',
                         facecolors='black', zorder=100, rasterized=False)
ax_morph_2.add_collection(polycol_2)


lines = []
lines_2 = []
line_names = []
line_names_2 = []
for i, elec_radius in enumerate(e_r):
    c = elec_clrs[elec_radius]

    if elec_radius > 1e-9:
        el = Ellipse((elec_1.x[0], elec_1.z[0]), elec_radius / 2, 2*elec_radius,
                 facecolor=c, zorder=-i)
        el_2 = Ellipse((elec_2.x[0], elec_2.z[0]), 2*elec_radius, elec_radius / 2,
                 facecolor=c, zorder=-i)
        ax_morph.add_artist(el)
        ax_morph_2.add_artist(el_2)
    else:
        ax_morph.plot(elec_1.x[0], elec_1.z[0], '.', c=c, ms=3)
        ax_morph_2.plot(elec_2.x[0], elec_2.z[0], '.', c=c, ms=3)

    l, = ax_eap_1.plot(cell.tvec, eaps_1[i], c=c)
    ax_eap_norm_1.plot(cell.tvec, eaps_1[i] / np.max(np.abs(eaps_1[i])), c=c)
    lines.append(l)
    line_names.append("$r$=%d µm" % elec_radius)
    
ax_morph.legend(lines, line_names,bbox_to_anchor=(-0.3, 0.47), loc='center left', borderaxespad=0)
simplify_axes(fig.axes)
fig_folder = 'figure_two_final'
fig_name = 'FIG2_elec1_and_2_new_radii'
plt.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}.png', bbox_inches = "tight") 